In [3]:
!pip install ultralytics pyyaml split-folders --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 whic

In [4]:
import os
import xml.etree.ElementTree as ET
import shutil
import splitfolders
from IPython.display import FileLink
from ultralytics import YOLO
import yaml

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# Define class mapping
CLASS_MAPPING = {
    'Horse': 0,
    'Serpent': 1,
    'Bird': 2,
    'Boar': 3,
    'Monkey': 4,
    'Ox': 5,
    'Ram': 6,
    'Tiger': 7,
    'Dog': 8,
    'Dragon': 9,
    'Hare': 10,
    'Rat': 11
}

# Create YOLO directory structure
dataset_dir = '/kaggle/working/dataset'
os.makedirs(f'{dataset_dir}/images', exist_ok=True)
os.makedirs(f'{dataset_dir}/labels', exist_ok=True)

In [7]:
def convert_xml_to_yolo(xml_path, output_dir):
    """Convert PASCAL VOC XML to YOLO format txt files"""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Get image dimensions
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    # Create label file
    txt_file = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_path))[0] + '.txt')
    
    with open(txt_file, 'w') as f:
        for obj in root.iter('object'):
            class_name = obj.find('name').text
            if class_name not in CLASS_MAPPING:
                continue  # Skip unknown classes
                
            cls = CLASS_MAPPING[class_name]
            
            # Get bounding box coordinates
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            # Convert to YOLO format (center x, center y, width, height)
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            
            f.write(f"{cls} {x_center} {y_center} {w} {h}\n")

In [8]:
# Process all images and annotations
input_dir = '/kaggle/input/narutooo/Images'

for filename in os.listdir(input_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(input_dir, filename)
        img_name = os.path.splitext(filename)[0] + '.jpg'
        img_path = os.path.join(input_dir, img_name)
        
        # Copy image if exists
        if os.path.exists(img_path):
            shutil.copy(img_path, f'{dataset_dir}/images/{img_name}')
            # Convert and save label
            convert_xml_to_yolo(xml_path, f'{dataset_dir}/labels')

In [9]:
# Split dataset (80% train, 20% validation)
splitfolders.ratio(
    dataset_dir,
    output="/kaggle/working/split_dataset",
    seed=42,
    ratio=(0.85, 0.15),
    group_prefix=None,
    move=False
)

Copying files: 232 files [00:00, 6136.04 files/s]


In [10]:
# Create dataset.yaml
yaml_content = """
path: /kaggle/working/split_dataset
train: train/images
val: val/images
names:
  0: Horse
  1: Serpent
  2: Bird
  3: Boar
  4: Monkey
  5: Ox
  6: Ram
  7: Tiger
  8: Dog
  9: Dragon
  10: Hare
  11: Rat
"""

with open('/kaggle/working/dataset.yaml', 'w') as f:
    f.write(yaml_content.strip())

In [11]:
import os
import shutil

# Step 1: Copy images and xmls to working directory
input_dir = '/kaggle/input/narutooo/Images'
working_dir = '/kaggle/working/Images'

# Create working dir
os.makedirs(working_dir, exist_ok=True)

# Copy all files
for filename in os.listdir(input_dir):
    src = os.path.join(input_dir, filename)
    dst = os.path.join(working_dir, filename)
    shutil.copy(src, dst)

print("✅ Files copied to working directory!")


✅ Files copied to working directory!


In [12]:
import os
import cv2
import xml.etree.ElementTree as ET

# Set your directory
input_dir = '/kaggle/working/Images'

# Loop over all images
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        img_path = os.path.join(input_dir, filename)
        xml_path = os.path.join(input_dir, os.path.splitext(filename)[0] + '.xml')
        
        # Read image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error reading {img_path}")
            continue
        
        height, width = img.shape[:2]
        
        # Flip image horizontally
        flipped_img = cv2.flip(img, 1)
        
        # Save flipped image
        flipped_img_name = os.path.splitext(filename)[0] + '_flipped.jpg'
        flipped_img_path = os.path.join(input_dir, flipped_img_name)
        cv2.imwrite(flipped_img_path, flipped_img)
        
        # Read XML annotation
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # Update bounding boxes
        for obj in root.findall('object'):
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            xmax = int(bbox.find('xmax').text)

            # Flip horizontally: new_xmin = width - old_xmax, new_xmax = width - old_xmin
            new_xmin = width - xmax
            new_xmax = width - xmin

            bbox.find('xmin').text = str(new_xmin)
            bbox.find('xmax').text = str(new_xmax)
        
        # Save flipped XML
        flipped_xml_name = os.path.splitext(filename)[0] + '_flipped.xml'
        flipped_xml_path = os.path.join(input_dir, flipped_xml_name)
        tree.write(flipped_xml_path)

print("✅ Done! Dataset doubled with flipped images and annotations.")


✅ Done! Dataset doubled with flipped images and annotations.


In [13]:
model = YOLO('yolov8n.pt')  # Using medium version for better accuracy


100%|██████████| 6.25M/6.25M [00:00<00:00, 83.2MB/s]


In [81]:
# # Classes that can be safely flipped horizontally
# flippable_classes = ['Horse','Bird', 'Serpent', 'Boar', 'Monkey', 'Ox', 'Ram', 'Dog','Tiger', 'Dragon', 'Hare', 'Rat']



In [82]:
# from albumentations import HorizontalFlip
# import cv2

# def augment_with_flips(image_path, label_path):
#     image = cv2.imread(image_path)
#     with open(label_path) as f:
#         boxes = [list(map(float, line.split())) for line in f]
    
#     # Apply horizontal flip
#     transformed = HorizontalFlip(p=1)(image=image, bboxes=boxes)
#     flipped_image = transformed['image']
#     flipped_boxes = transformed['bboxes']
    
#     # Adjust YOLO format coordinates for flipped boxes
#     for box in flipped_boxes:
#         cls, x_center, y_center, width, height = box
#         new_x_center = 1.0 - x_center  # Mirror horizontally
#         flipped_box = [cls, new_x_center, y_center, width, height]
    
#     return flipped_image, flipped_boxes

In [84]:
# # Only augment flippable classes
# for img_path, label_path in dataset:
#     with open(label_path) as f:
#         classes_present = [int(line.split()[0]) for line in f]
    
#     if any(class_names[cls] in flippable_classes for cls in classes_present):
#         flipped_img, flipped_labels = augment_with_flips(img_path, label_path)
#         save_augmented_data(flipped_img, flipped_labels)

In [15]:
results = model.train(
    data='/kaggle/working/dataset.yaml',
    epochs=200,              # 🧠 300 is a lot, risking overfitting. 200 is safer with EarlyStopping
    batch=8,                 # 🚀 Now you doubled your dataset (232 images), batch=8 is better
    imgsz=672,
    optimizer='AdamW',       # ✅ Great choice
    lr0=2e-4,                # ⚡ Smaller initial LR for better convergence (0.0002)
    lrf=0.01,                # ✅ Keep final LR small
    cos_lr=True,             # ✅ Best choice for small datasets
    weight_decay=0.0001,     # 🔥 Reduce it to 0.0001 (less aggressive, small data needs it)
    warmup_epochs=3,         # 🧠 5 is too long for 232 images, 3 is enough
    warmup_momentum=0.7,                # ⚡ Fine-tune warmup
    warmup_bias_lr=0.1, 
    label_smoothing=0.10,    # 🚀 Push label smoothing a bit higher for small, noisy datasets
    # ema=True,                          # ✅ Add EMA

    # ⚡ Augmentation (very balanced now)
    hsv_h=0.015,             
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,              # 🧠 Lower rotation (small datasets prefer lower transforms)
    translate=0.1,
    scale=0.4,
    shear=0.0,
    perspective=0.0005,
    flipud=0.3,
    fliplr=0.5,
    mosaic=0.4,              # 🧠 Reduced mosaic to 40% (mosaic can destroy small data if too high)
    mixup=0.05,              # 🧠 Even lower mixup, safe (small datasets hate mixup too much)
    copy_paste=0.0,          # ✅ Good to disable

    # ⚙️ Class & Regularization
    cls=1.0,                 # 🧠 Keep class loss weight = 1 (1.5 is a bit aggressive)
    dropout=0.1,             # ✅ Good amount

    # 📈 Validation settings
    split='val',
    val=True,
    patience=30,             # 🚀 Shorten patience to 20 (don't let overfit 30+ epochs)
    device=0,
    project='/kaggle/working/results',
    name='naruto_super_optimized',
    
    # 📊 Debugging & Saving
    plots=True,
    save=True,
    exist_ok=True
)


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.118 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset.yaml, epochs=200, time=None, patience=30, batch=8, imgsz=672, save=True, save_period=-1, cache=False, device=0, workers=8, project=/kaggle/working/results, name=naruto_super_optimized, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

100%|██████████| 755k/755k [00:00<00:00, 17.8MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 74.3MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1807.4±530.3 MB/s, size: 82.9 KB)


...


99 epochs completed in 0.049 hours.
Optimizer stripped from /kaggle/working/results/naruto_super_optimized/weights/last.pt, 6.3MB
Optimizer stripped from /kaggle/working/results/naruto_super_optimized/weights/best.pt, 6.3MB

Validating /kaggle/working/results/naruto_super_optimized/weights/best.pt...
Ultralytics 8.3.118 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 12.40it/s]


                   all         18         18      0.907          1      0.995      0.764
                 Horse          1          1      0.949          1      0.995      0.995
                  Bird          2          2      0.858          1      0.995      0.689
                  Boar          4          4      0.925          1      0.995      0.594
                    Ox          2          2      0.885          1      0.995      0.847
                   Ram          2          2      0.873          1      0.995      0.722
                 Tiger          3          3      0.924          1      0.995      0.735
                   Dog          1          1      0.816          1      0.995      0.697
                Dragon          2          2      0.976          1      0.995      0.796
                   Rat          1          1      0.961          1      0.995      0.796


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.3ms preprocess, 2.5ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /kaggle/working/results/naruto_super_optimized


In [16]:
# Export best model
best_model = '/kaggle/working/results/naruto_super_optimized/weights/best.pt'
shutil.copy(best_model, '/kaggle/working/best_naruto_signs_8.pt')

# Create downloadable link
FileLink('/kaggle/working/best_naruto_signs_8.pt')

/kaggle/working/best_naruto_signs_8.pt

In [17]:
# Verification test with class labels
test_model = YOLO('/kaggle/working/best_naruto_signs_8.pt')

# Get class names from YAML
with open('/kaggle/working/dataset.yaml') as f:
    data = yaml.safe_load(f)
    class_names = data['names']

results = test_model.predict(
    source='/kaggle/working/split_dataset/val/images',
    save=True,
    conf=0.5,
    line_thickness=2,
    show_labels=True,
    show_conf=True
)

print("\nClass distribution in validation set:")
for class_id, name in class_names.items():
    count = 0
    for r in results:
        if r.boxes.cls.nelement() > 0:  # Check if any detections exist
            detected_classes = r.boxes.cls.int().tolist()
            count += detected_classes.count(class_id)
    print(f"{name}: {count} detections")

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in in the future. Use 'line_width' instead.

image 1/18 /kaggle/working/split_dataset/val/images/Bird.20fa2833-da6c-11ec-a6e5-9cfce8f20ad4.jpg: 512x672 1 Bird, 40.8ms
image 2/18 /kaggle/working/split_dataset/val/images/Bird.23fac499-da6c-11ec-9117-9cfce8f20ad4.jpg: 512x672 2 Birds, 7.4ms
image 3/18 /kaggle/working/split_dataset/val/images/Boar.3ca7088d-dff3-11ec-ae62-9cfce8f20ad4.jpg: 512x672 1 Boar, 7.4ms
image 4/18 /kaggle/working/split_dataset/val/images/Boar.3f14da29-dff3-11ec-a1ed-9cfce8f20ad4.jpg: 512x672 1 Boar, 7.4ms
image 5/18 /kaggle/working/split_dataset/val/images/Boar.404b946f-dff3-11ec-a9dd-9cfce8f20ad4.jpg: 512x672 1 Boar, 7.4ms
image 6/18 /kaggle/working/split_dataset/val/images/Boar.43f0a86b-dff3-11ec-af36-9cfce8f20ad4.jpg: 512x672 1 Boar, 7.3ms
image 7/18 /kaggle/working/split_dataset/val/images/Dog.2a8792fc-dff6-11ec-aaa2-9cfce8f20ad4.jpg: 512x672 1 Dog, 7.3ms
image 8/18 /kaggle/working/split_dataset/val/i

> 